## Read in NOAA Daily Weather Data for 2023 for all Groundstations

- Pull data directly from NOAA Daily Weather Data
- Group by station & date for TMAX, TMIN, TAVG and PRCP
- Fix NaN Values for Temps and PRCP
- Fix the scale on the Temp data for 1/10 deg celcius to deg Fahrenheit.
- Fix the scale on the PRCP (precipitaiton) for 1/10 mm to inches
- This is 37,402,838 records and occasionally crashes Jupyter Notebooks
    - Therefore, save resulting DataFrame to a parquet file

Perform this cleaning in two ways
1. Adjusted TMAX: Fix all NaN values for TMAX with an algorithm.  Therefore all sites will have a TMAX value if missing.
    - results written to: us_ca_daily_weather.parquet.gzip 
2. Limited TMAX: Drop all records with NaN values for TMAX (limited results)
    - results written to: us_ca_daily_weather_limited_TMAX.parquet.gzip

Note:  method 2 is preferred because we are basing analysis on TMAX.  Filling wtih TMIN or TAVG as in method 1 will skew the data make our analysis error prone.


In [1]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp
import os

### Insure result_files directory exists

In [2]:
def create_result_files_subdirectory():
    # Define the directory path
    directory = "result_files"
    
    # Check if the directory already exists
    if not os.path.exists(directory):
        # Create the directory if it doesn't exist
        os.makedirs(directory)
        print(f"Directory '{directory}' created successfully.")
    else:
        print(f"Directory '{directory}' already exists.")

# Call the function to create the directory
create_result_files_subdirectory()

Directory 'result_files' already exists.


### Previously we copied the data to a local source_files directory and read from there
  `weather_data = pd.read_csv('source_files/weather/2023.csv.gz', compression='gzip', header=None)`

#### However, we updated the method below to read directly from NOAA.

In [3]:



# read daily weather data for all NOAA stations for an entire year
noaa_daily_site = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/'
noaa_daily_2023 = noaa_daily_site + 'by_year/2023.csv.gz'

weather_data = pd.read_csv(noaa_daily_2023, compression='gzip', header=None) 
weather_data

,0,1,2,3,4,5,6,7
0,AE000041196,20230101,TMAX,252,NaN,NaN,S,NaN
1,AE000041196,20230101,TMIN,149,NaN,NaN,S,NaN
2,AE000041196,20230101,PRCP,0,D,NaN,S,NaN
3,AE000041196,20230101,TAVG,207,H,NaN,S,NaN
4,AEM00041194,20230101,TMAX,255,NaN,NaN,S,NaN
...,...,...,...,...,...,...,...,...
37320792,ZI000067775,20231231,TMIN,176,NaN,NaN,S,NaN
37320793,ZI000067775,20231231,TAVG,216,H,NaN,S,NaN
37320794,ZI000067975,20231231,TMIN,169,NaN,NaN,S,NaN
37320795,ZI000067975,20231231,PRCP,10,NaN,NaN,S,NaN


In [4]:
# Drop unecessary Columns 4-7
weather_data.drop([4, 5, 6, 7], axis=1, inplace=True)
weather_data.columns

Index([0, 1, 2, 3], dtype='int64')

In [5]:
# rename columns to be more descriptive
weather_data.rename(columns={0: 'ID', 1: 'Date', 2: 'Attr', 3: 'Val'}, inplace=True)
weather_data

,ID,Date,Attr,Val
0,AE000041196,20230101,TMAX,252
1,AE000041196,20230101,TMIN,149
2,AE000041196,20230101,PRCP,0
3,AE000041196,20230101,TAVG,207
4,AEM00041194,20230101,TMAX,255
...,...,...,...,...
37320792,ZI000067775,20231231,TMIN,176
37320793,ZI000067775,20231231,TAVG,216
37320794,ZI000067975,20231231,TMIN,169
37320795,ZI000067975,20231231,PRCP,10


In [6]:
# How many unique values in each column
unique_counts = weather_data.nunique()
unique_counts

ID      42031
Date      365
Attr       71
Val      5021
dtype: int64

There are 42027 weather stations globally in the dataset


In [7]:
# Retain only US and CA weather data
weather_US_CA = weather_data[weather_data['ID'].str.contains("^US|^CA")]
weather_US_CA

,ID,Date,Attr,Val
5975,CA001011500,20230101,TMAX,80
5976,CA001011500,20230101,TMIN,20
5977,CA001011500,20230101,PRCP,0
5978,CA001011500,20230101,SNOW,0
5979,CA001011500,20230101,SNWD,0
...,...,...,...,...
37320525,USW00096408,20231231,TMIN,-178
37320526,USW00096408,20231231,PRCP,0
37320527,USW00096409,20231231,TMAX,-195
37320528,USW00096409,20231231,TMIN,-309


In [8]:
# How many unique values in each column
unique_counts = weather_US_CA.nunique()
unique_counts

ID      32260
Date      365
Attr       66
Val      4508
dtype: int64

In [9]:
weather_readings_available = weather_US_CA.pivot(index=['ID', 'Date'], columns=['Attr'], values='Val').count().sort_values(ascending=False).head(20)
weather_readings_available

Attr
PRCP    8305960
SNOW    5442850
TMAX    2941479
TMIN    2938071
SNWD    2607730
TOBS    1579413
TAVG    1127018
WESD     519826
AWND     415406
WSF2     393850
WDF2     393738
WSF5     380268
WDF5     379531
WESF     321969
WT01     169595
RHMN     151573
RHMX     151573
RHAV     151520
ADPT     150987
AWBT     150987
dtype: int64

### Useful readings

- There are 32259 unique weather stations in US and CA combined

- Useful readings are PRCP, TMAX, TMIN, TAVG
    - PRCP is precipitation (snow, rain, etc) in tenths of mm
    - TMAX is max daily temp in tenths of degress C
    - TMIN is min daily temp in tenths of degress C
    - TAVG is average daily temp in tenths of degress C (mean of hourly temps)

- Not useful readings
    - Snow related readings (SNOW, SNWD, WESD, WESF) is incorporated in PRCP
    - AWND is average daily wind.  Does not provide max and can hide extremes
    - Other readings do not provide enough data

#### Collect data on PRCP, TMAX, TMIN and TAVG

In [10]:
# Keep only rows with an Attribute of 'TMAX', 'TMIN', 'TAVG', 'PRCP'
weather_US_CA_Temps = weather_US_CA[weather_US_CA['Attr'].str.contains("TMAX|TAVG|TMIN|PRCP")]
weather_US_CA_Temps

,ID,Date,Attr,Val
5975,CA001011500,20230101,TMAX,80
5976,CA001011500,20230101,TMIN,20
5977,CA001011500,20230101,PRCP,0
5980,CA001011500,20230101,TAVG,50
5981,CA001012475,20230101,TMAX,70
...,...,...,...,...
37320525,USW00096408,20231231,TMIN,-178
37320526,USW00096408,20231231,PRCP,0
37320527,USW00096409,20231231,TMAX,-195
37320528,USW00096409,20231231,TMIN,-309


### Pivot Data for each station ID and Date

Each row is a unique station ID/date pair with TMAX, TMIN, TAVG & PRCP

In [11]:
# Combine all rows with the same ID and Date.  Creating a column for each attribute.
weather_data_df = weather_US_CA_Temps.pivot(index=['ID', 'Date'], columns=['Attr'], values='Val')
weather_data_df


Attr                  PRCP  TAVG   TMAX   TMIN
ID          Date                              
CA001011500 20230101   0.0  50.0   80.0   20.0
            20230102   2.0  50.0   70.0   30.0
            20230103  12.0  43.0   55.0   30.0
            20230104  16.0  48.0   65.0   30.0
            20230105  17.0  70.0   90.0   50.0
...                    ...   ...    ...    ...
USW00096409 20231227   0.0   NaN -125.0 -285.0
            20231228   0.0   NaN -145.0 -209.0
            20231229   0.0   NaN -186.0 -255.0
            20231230   0.0   NaN -230.0 -265.0
            20231231   0.0   NaN -195.0 -309.0

[8842869 rows x 4 columns]

In [12]:
# Reset the index so that Station ID is now a column
weather_data_df.reset_index(inplace=True)

In [13]:
# Show column types
weather_data_df.dtypes


Attr
ID       object
Date      int64
PRCP    float64
TAVG    float64
TMAX    float64
TMIN    float64
dtype: object

In [14]:
# convert the 'Date' column to datetime format
weather_data_df['Date']= pd.to_datetime(weather_data_df['Date'], format='%Y%m%d')
weather_data_df.dtypes


Attr
ID              object
Date    datetime64[ns]
PRCP           float64
TAVG           float64
TMAX           float64
TMIN           float64
dtype: object

In [15]:
#Show cleaned data
weather_data_df

Attr,ID,Date,PRCP,TAVG,TMAX,TMIN
0,CA001011500,2023-01-01,0.0,50.0,80.0,20.0
1,CA001011500,2023-01-02,2.0,50.0,70.0,30.0
2,CA001011500,2023-01-03,12.0,43.0,55.0,30.0
3,CA001011500,2023-01-04,16.0,48.0,65.0,30.0
4,CA001011500,2023-01-05,17.0,70.0,90.0,50.0
...,...,...,...,...,...,...
8842864,USW00096409,2023-12-27,0.0,NaN,-125.0,-285.0
8842865,USW00096409,2023-12-28,0.0,NaN,-145.0,-209.0
8842866,USW00096409,2023-12-29,0.0,NaN,-186.0,-255.0
8842867,USW00096409,2023-12-30,0.0,NaN,-230.0,-265.0


### Create functions to fix NaN values in each column

In [16]:
# Create functions to fix NaN values in each column
# Since TMAX NaN dropped, that function will not make changes.

# Fix TMIN Column
def fix_min(tmin,tmax,tavg):
#    print(f"MIN: {tmin}  MAX: {tmax}  AVG: {tavg}")
    if np.isnan(tmin):
        if np.isnan(tavg):
            if np.isnan(tmax):
                return 0
            else:
                return tmax
        else:
            return tavg
        return tmax
    else:
        return tmin

# Fix TMAX Column
def fix_max(tmin,tmax,tavg):
#    print(f"MIN: {tmin}  MAX: {tmax}  AVG: {tavg}")
    if np.isnan(tmax):
        if np.isnan(tavg):
            if np.isnan(tmin):
                return 0
            else:
                return tmin
        else:
            return tavg
        return tmax
    else:
        return tmax

# Fix TAVG Column
def fix_avg(tmin,tmax,tavg):
#    print(f"MIN: {tmin}  MAX: {tmax}  AVG: {tavg}")
    if np.isnan(tavg):
        if np.isnan(tmin):
            if np.isnan(tmax):
                return 0
            else:
                return tmax
        else:
            if np.isnan(tmax):
                return tmin
            else:
                return (tmin + tmax) / 2
    else:
        return tavg

### Fix and save weather data (adjusting TMAX)
- Fix Non Values in TMIN, TMAX, TAVG and PRCP columns
- Convert units to Deg Farhenheit and Inches Precipitation
- Save weather_data_df to result_files/us_ca_daily_weather.parquet.gzip

In [17]:
# copy weather_data_df to weather_data_adju_tmax
weather_data_adju_tmax = weather_data_df.copy()

In [18]:
# Fix NaN values in each column

pd.options.mode.chained_assignment = None  # default='warn'

weather_data_adju_tmax['TMAX'] = weather_data_adju_tmax.apply(lambda x: fix_max(x['TMIN'], x['TMAX'], x['TAVG']), axis=1)

weather_data_adju_tmax['TMIN'] = weather_data_adju_tmax.apply(lambda x: fix_min(x['TMIN'], x['TMAX'], x['TAVG']), axis=1)

weather_data_adju_tmax['TAVG'] = weather_data_adju_tmax.apply(lambda x: fix_avg(x['TMIN'], x['TMAX'], x['TAVG']), axis=1)

weather_data_adju_tmax['PRCP'] = weather_data_adju_tmax['PRCP'].fillna(0)

weather_data_adju_tmax

Attr,ID,Date,PRCP,TAVG,TMAX,TMIN
0,CA001011500,2023-01-01,0.0,50.0,80.0,20.0
1,CA001011500,2023-01-02,2.0,50.0,70.0,30.0
2,CA001011500,2023-01-03,12.0,43.0,55.0,30.0
3,CA001011500,2023-01-04,16.0,48.0,65.0,30.0
4,CA001011500,2023-01-05,17.0,70.0,90.0,50.0
...,...,...,...,...,...,...
8842864,USW00096409,2023-12-27,0.0,-205.0,-125.0,-285.0
8842865,USW00096409,2023-12-28,0.0,-177.0,-145.0,-209.0
8842866,USW00096409,2023-12-29,0.0,-220.5,-186.0,-255.0
8842867,USW00096409,2023-12-30,0.0,-247.5,-230.0,-265.0


In [19]:
# Convert from tenths deg Celcius to deg Fahrenheit (x/10 * (9/5 +32))
# Conver PRCP from tenths of mm to inches of precipitation (x/10/25.4)

weather_data_adju_tmax['TMAX'] = weather_data_adju_tmax['TMAX'].apply(lambda x: x /10 * 9 / 5 + 32)

weather_data_adju_tmax['TMIN'] = weather_data_adju_tmax['TMIN'].apply(lambda x: x /10 * 9 / 5 + 32)

weather_data_adju_tmax['TAVG'] = weather_data_adju_tmax['TAVG'].apply(lambda x: x /10 * 9 / 5 + 32)

# Convert from mm to inches
weather_data_adju_tmax['PRCP'] = weather_data_adju_tmax['PRCP'].apply(lambda x: x / 254)

In [20]:
# Write weather data to parquet file
weather_data_adju_tmax.to_parquet('result_files/stp1_us_ca_daily_weather_TMAX_adjusted.parquet.gzip', compression='gzip', engine="fastparquet")  

### Create a weather_limited_tmax_df with rows of only valid TMAX readings
- Drop rows with NaN values in TMAX
- Fix Non Values in TMIN, TMAX, TAVG and PRCP columns
- Convert units to Deg Farhenheit and Inches Precipitation
- Save weather_data_df to result_files/us_ca_daily_weather.parquet.gzip

In [21]:
# Drop all rows with missing TMAX value

weather_limited_tmax_df = weather_data_df.dropna(subset = ['TMAX'])
weather_limited_tmax_df

Attr,ID,Date,PRCP,TAVG,TMAX,TMIN
0,CA001011500,2023-01-01,0.0,50.0,80.0,20.0
1,CA001011500,2023-01-02,2.0,50.0,70.0,30.0
2,CA001011500,2023-01-03,12.0,43.0,55.0,30.0
3,CA001011500,2023-01-04,16.0,48.0,65.0,30.0
4,CA001011500,2023-01-05,17.0,70.0,90.0,50.0
...,...,...,...,...,...,...
8842864,USW00096409,2023-12-27,0.0,NaN,-125.0,-285.0
8842865,USW00096409,2023-12-28,0.0,NaN,-145.0,-209.0
8842866,USW00096409,2023-12-29,0.0,NaN,-186.0,-255.0
8842867,USW00096409,2023-12-30,0.0,NaN,-230.0,-265.0


In [22]:
# Fix NaN values in each column
# No need to fix TMAX

pd.options.mode.chained_assignment = None  # default='warn'

#df['TMAX'] = df.apply(lambda x: fix_max(x['TMIN'], x['TMAX'], x['TAVG']), axis=1)

weather_limited_tmax_df['TMIN'] = weather_limited_tmax_df.apply(lambda x: fix_min(x['TMIN'], x['TMAX'], x['TAVG']), axis=1)

weather_limited_tmax_df['TAVG'] = weather_limited_tmax_df.apply(lambda x: fix_avg(x['TMIN'], x['TMAX'], x['TAVG']), axis=1)

weather_limited_tmax_df['PRCP'] = weather_limited_tmax_df['PRCP'].fillna(0)

weather_limited_tmax_df

Attr,ID,Date,PRCP,TAVG,TMAX,TMIN
0,CA001011500,2023-01-01,0.0,50.0,80.0,20.0
1,CA001011500,2023-01-02,2.0,50.0,70.0,30.0
2,CA001011500,2023-01-03,12.0,43.0,55.0,30.0
3,CA001011500,2023-01-04,16.0,48.0,65.0,30.0
4,CA001011500,2023-01-05,17.0,70.0,90.0,50.0
...,...,...,...,...,...,...
8842864,USW00096409,2023-12-27,0.0,-205.0,-125.0,-285.0
8842865,USW00096409,2023-12-28,0.0,-177.0,-145.0,-209.0
8842866,USW00096409,2023-12-29,0.0,-220.5,-186.0,-255.0
8842867,USW00096409,2023-12-30,0.0,-247.5,-230.0,-265.0


In [23]:
# Convert from tenths deg Celcius to deg Fahrenheit (x/10 * (9/5 +32))
# Conver PRCP from tenths of mm to inches of precipitation (x/10/25.4)

weather_limited_tmax_df['TMAX'] = weather_limited_tmax_df['TMAX'].apply(lambda x: x /10 * 9 / 5 + 32)

weather_limited_tmax_df['TMIN'] = weather_limited_tmax_df['TMIN'].apply(lambda x: x /10 * 9 / 5 + 32)

weather_limited_tmax_df['TAVG'] = weather_limited_tmax_df['TAVG'].apply(lambda x: x /10 * 9 / 5 + 32)

# Convert from mm to inches
weather_limited_tmax_df['PRCP'] = weather_limited_tmax_df['PRCP'].apply(lambda x: x / 254)

In [24]:
# Write limited TMAX weather data to parquet file
weather_limited_tmax_df.to_parquet('result_files/stp1_us_ca_daily_weather_TMAX_limited.parquet.gzip', compression='gzip', engine="fastparquet")  


In [25]:
# Read weather paqquet file into a pandas dataframe to verify data integrity is maintained
# Read with adjusted TMAX
weather_data_adju_tmax = pd.read_parquet('result_files/stp1_us_ca_daily_weather_TMAX_adjusted.parquet.gzip', engine="fastparquet") 

#Read with limited TMAX
weather_limited_tmax_df = pd.read_parquet('result_files/stp1_us_ca_daily_weather_TMAX_limited.parquet.gzip', engine="fastparquet")  


In [26]:
# Check integrity of adjusted TMAX weatherdata
weather_data_adju_tmax.dtypes


Attr
ID              object
Date    datetime64[ns]
PRCP           float64
TAVG           float64
TMAX           float64
TMIN           float64
dtype: object

In [27]:
# Check integrity of limited TMAX data
weather_limited_tmax_df.dtypes

Attr
ID              object
Date    datetime64[ns]
PRCP           float64
TAVG           float64
TMAX           float64
TMIN           float64
dtype: object

In [28]:
weather_data_adju_tmax

Attr,ID,Date,PRCP,TAVG,TMAX,TMIN
0,CA001011500,2023-01-01,0.000000,41.00,46.40,35.60
1,CA001011500,2023-01-02,0.007874,41.00,44.60,37.40
2,CA001011500,2023-01-03,0.047244,39.74,41.90,37.40
3,CA001011500,2023-01-04,0.062992,40.64,43.70,37.40
4,CA001011500,2023-01-05,0.066929,44.60,48.20,41.00
...,...,...,...,...,...,...
8842864,USW00096409,2023-12-27,0.000000,-4.90,9.50,-19.30
8842865,USW00096409,2023-12-28,0.000000,0.14,5.90,-5.62
8842866,USW00096409,2023-12-29,0.000000,-7.69,-1.48,-13.90
8842867,USW00096409,2023-12-30,0.000000,-12.55,-9.40,-15.70


In [29]:
weather_limited_tmax_df

Attr,ID,Date,PRCP,TAVG,TMAX,TMIN
index,,,,,,
0,CA001011500,2023-01-01,0.000000,41.00,46.40,35.60
1,CA001011500,2023-01-02,0.007874,41.00,44.60,37.40
2,CA001011500,2023-01-03,0.047244,39.74,41.90,37.40
3,CA001011500,2023-01-04,0.062992,40.64,43.70,37.40
4,CA001011500,2023-01-05,0.066929,44.60,48.20,41.00
...,...,...,...,...,...,...
8842864,USW00096409,2023-12-27,0.000000,-4.90,9.50,-19.30
8842865,USW00096409,2023-12-28,0.000000,0.14,5.90,-5.62
8842866,USW00096409,2023-12-29,0.000000,-7.69,-1.48,-13.90
